In [ ]:
""""Step 1: Import Libraries and Load Environment Variables

In this step, I set up the project by importing necessary libraries and securely loading API keys from an external file:

Library Imports:

dotenv & os: Load sensitive API keys from Astra.env.
openai: For generating dynamic responses.
speech_recognition & pyttsx3: For capturing voice input and converting text to speech.
time & random: For delays and selecting random responses.
requests: For HTTP requests to external APIs (like weather data).
API Key Loading:
The load_dotenv("Astra.env") call reads API keys from the file, which are then accessed using os.getenv(). This keeps my credentials secure and separate from the code.

API Key Configuration:
I set the OpenAI API key by assigning it to openai.api_key for later use in the project"""

In [2]:
from dotenv import load_dotenv
import os
import openai
import speech_recognition as sr
import pyttsx3
import time
import random
import requests

# Load API keys from Astra.env
load_dotenv("Astra.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
openweather_api_key = os.getenv("OPENWEATHER_API_KEY")

# Set OpenAI API key
openai.api_key = openai_api_key


In [3]:
"""Step 2: Initialize Speech Recognizer and Text-to-Speech Engine

In this step, I set up the core components for voice input and output:

Speech Recognizer:
I initialize sr.Recognizer() to capture and process audio from the microphone.

Text-to-Speech Engine:
I initialize pyttsx3 for converting text responses into spoken words and adjust the speaking rate to 150 for clear and natural-sounding output"""

In [4]:
# Initialize the speech recognizer and text-to-speech engine
recognizer = sr.Recognizer()
tts_engine = pyttsx3.init()
tts_engine.setProperty('rate', 150)  # Adjust speaking rate if needed


In [5]:
###Define Helper Functions

In [6]:
"""Step 3: Text-to-Speech and Listening Functions

speak(text):
Prints and speaks the given text aloud using the TTS engine.

listen():
Listens via the microphone, adjusts for ambient noise, and converts your speech into lowercase text. It gracefully handles errors if the speech isn’t understood"""

In [7]:
def speak(text):
    """Speak the provided text and print it for visual feedback."""
    print("Astra:", text)
    tts_engine.say(text)
    tts_engine.runAndWait()

def listen():
    """Listen for a command from the user and return it as lowercase text."""
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        audio = recognizer.listen(source)
    try:
        command = recognizer.recognize_google(audio)
        print("You said:", command)
        return command.lower()
    except sr.UnknownValueError:
        print("Sorry, I did not understand that.")
        return ""
    except sr.RequestError:
        print("Could not request results from the service.")
        return ""


In [8]:
"""Step 4: General AI Question Answering

Functionality:
The answer_ai_question(question) function sends your query to OpenAI's ChatCompletion API using the gpt-3.5-turbo model, then extracts and returns the generated answer.

Error Handling:
If any error occurs, it prints the error details and returns a fallback message"""

In [9]:
def answer_ai_question(question):
    """Answer a general question using OpenAI's ChatCompletion API with gpt-3.5-turbo."""
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a knowledgeable AI assistant."},
                {"role": "user", "content": question}
            ],
            max_tokens=150,
            temperature=0.7
        )
        answer = response.choices[0].message['content'].strip()
        return answer
    except Exception as e:
        print("Error details:", e)
        return "Sorry, I couldn't process your question at the moment."


In [10]:
"""Step 5: Weather Update Function

Functionality:
The get_weather(city) function builds a URL using the provided city name and your OpenWeatherMap API key, then fetches the current weather data.

Response:
If successful, it returns a formatted string with the temperature and weather description; otherwise, it returns an error message"""

In [11]:
def get_weather(city):
    """Return the current weather for a given city using OpenWeatherMap."""
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = f"{base_url}q={city}&appid={openweather_api_key}&units=metric"
    try:
        response = requests.get(complete_url)
        if response.status_code == 200:
            data = response.json()
            temp = data['main']['temp']
            description = data['weather'][0]['description']
            return f"The temperature in {city} is {temp}°C with {description}."
        else:
            return "Sorry, I could not retrieve the weather information for that city."
    except Exception as e:
        print("Weather API error:", e)
        return "An error occurred while fetching the weather data."


In [12]:
"""Step 6: Fun Interactive Query Function

Functionality:
The fun_interaction() function randomly selects and returns a joke or fun fact from a predefined list"""

In [13]:
def fun_interaction():
    """Return a fun interactive response (joke or fun fact)."""
    jokes = [
        "Why do programmers prefer dark mode? Because light attracts bugs!",
        "I would tell you a joke about UDP, but you might not get it!",
        "Why did the computer show up at work late? It had a hard drive!"
    ]
    return random.choice(jokes)


In [14]:
"""Step 7: Reminder Management Functions

add_reminder(reminder_text):
Adds the given reminder to a list and returns a confirmation message.

show_reminders():
Returns a summary of all saved reminders, or a message if none exist"""


In [15]:
reminders = []

def add_reminder(reminder_text):
    """Add a reminder to the list."""
    reminders.append(reminder_text)
    return "Reminder added."

def show_reminders():
    """Return a summary of all saved reminders."""
    if reminders:
        return "Your reminders: " + "; ".join(reminders)
    else:
        return "You have no reminders."


In [16]:
"""Step 8: Process Commands

Purpose:
This function interprets the user's voice command and calls the corresponding functionality.

How It Works:

If the command contains "weather", it prompts for a city name, fetches the weather data, and speaks the result.
If "add reminder" is detected, it asks for a reminder, adds it to the list, and confirms.
If the user says "show reminders", it lists all saved reminders.
If "fun" is mentioned, it delivers a random joke or fun fact.
The command "exit" terminates the assistant with a farewell message.
For any other command, it uses the AI question-answering function to generate a response"""

In [17]:
def process_command(command):
    if "weather" in command:
        speak("Which city's weather would you like to know?")
        city = listen()
        if city:
            speak(get_weather(city))
        else:
            speak("I didn't catch the city name.")
    
    elif "add reminder" in command:
        speak("What reminder would you like to add?")
        reminder_text = listen()
        if reminder_text:
            speak(add_reminder(reminder_text))
        else:
            speak("I didn't catch that. Please try again.")
    
    elif "show reminders" in command:
        speak(show_reminders())
    
    elif "fun" in command:
        speak(fun_interaction())
    
    elif "exit" in command:
        speak("Goodbye! Have a great day!")
        return False  # Signal to exit the loop
    
    elif command.strip() != "":
        answer = answer_ai_question(command)
        speak(answer)
    
    else:
        speak("I did not catch that. Please try again.")
    
    return True  # Continue the loop


In [18]:
"""Step 9: Main Loop

Functionality:
The astra_assistant() function initiates the assistant by greeting the user and then enters a loop where it waits for the user to trigger a command, listens for input, processes that command, and repeats until the user says "exit."

Workflow:

Greets the user with a welcome message.
Waits for the user to press Enter to start listening.
Captures and processes the spoken command via the process_command() function.
Continues looping, with a brief pause between iterations for clarity"""

In [19]:
def astra_assistant():
    speak("Hello, I am Astra. How can I assist you today?")
    running = True
    while running:
        input("Press Enter to speak your command...")  # Trigger listening
        command = listen()
        running = process_command(command)
        time.sleep(1)  # Brief pause for clarity




In [20]:
"""Step 10: Calling Our Assistant

What It Does:
This line simply starts the voice assistant by calling the astra_assistant() function.

Purpose:
It launches the interactive session where Astra listens for your commands and responds accordingly until you say "exit""

In [22]:
# Run the assistant
astra_assistant()

Astra: Hello, I am Astra. How can I assist you today?


Press Enter to speak your command... 


Listening...
You said: capital of France
Astra: The capital of France is Paris.


Press Enter to speak your command... 


Listening...
You said: what is blockchain
Astra: Blockchain is a decentralized, distributed ledger technology that records transactions across a network of computers. Each transaction is stored in a "block," which is then linked to the previous block, forming a chain of blocks - hence the name "blockchain." This technology is known for its security, transparency, and immutability, making it a popular choice for applications such as cryptocurrencies, supply chain management, voting systems, and more.


...

Listening...
Sorry, I did not understand that.
Astra: I did not catch that. Please try again.


Press Enter to speak your command... 


Listening...
You said: f u n Fun
Astra: Why did the computer show up at work late? It had a hard drive!


Press Enter to speak your command... 


Listening...
You said: exit
Astra: Goodbye! Have a great day!
